In [ ]:
!pip install -q transformers accelerate huggingface_hub



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.8 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login("YOUR_SECRET_TOKEN_HERE")  # ← replace with your Hugging Face access token


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    token=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def generate_questions(job_title):
    prompt = f"You are an HR assistant. Generate 3 interview questions for the job role of {job_title}. Only output the questions, numbered 1 to 3."

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        pad_token_id=tokenizer.eos_token_id  # Important to avoid generation error
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text


In [ ]:
job_title = "Data Analyst"
questions = generate_questions(job_title)
print("Interview Questions:\n")
print(questions)


Interview Questions:

You are an HR assistant. Generate 3 interview questions for the job role of Data Analyst. Only output the questions, numbered 1 to 3. 

Here are your 3 interview questions:

1. Can you walk me through a time when you had to analyze a complex dataset and identify key trends or insights? How did you approach the task, and what were some of the key findings you discovered?
2. How do you stay up-to-date with new tools and technologies in the field of data analysis, and can you give me an example of a new skill you've learned recently and how you've applied it in your work?
3. Suppose you're tasked with creating a dashboard to track key performance indicators for a company. How would you approach the design of the dashboard, and what metrics would you include to ensure that it's effective in communicating insights to stakeholders? 


In [ ]:
# STEP 6: Function to evaluate user answers
def evaluate_answers(job_title, questions, answers):
    combined = ""
    for i in range(3):
        combined += f"Q{i+1}: {questions[i]}\nA{i+1}: {answers[i]}\n"

    prompt = (
        f"You are a technical interviewer. Based on the following answers for the job title '{job_title}', "
        f"give feedback on the strengths and weaknesses. Also, give a suitability percentage out of 100.\n\n"
        f"{combined}"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# STEP 7: Run the full interaction
job = input("Enter the job title: ")
print("\nGenerating questions...\n")
questions_text = generate_questions(job)
questions_lines = [line for line in questions_text.split("\n") if line.strip().startswith("1") or line.strip().startswith("2") or line.strip().startswith("3")]
print("\n".join(questions_lines))

answers = []
for i in range(3):
    ans = input(f"\nYour answer to Q{i+1}: ")
    answers.append(ans)

print("\nEvaluating your answers...\n")
feedback = evaluate_answers(job, questions_lines, answers)
print(feedback)

Enter the job title: Software Engineer

Generating questions...

1. Can you describe your experience with [specific programming language or technology]? How have you applied it in previous projects?
2. How do you approach debugging and troubleshooting complex software issues? Can you give an example of a particularly challenging problem you've solved in the past?
3. How do you prioritize and manage multiple projects or tasks simultaneously? Can you walk me through your process for staying organized and meeting deadlines?

Your answer to Q1: I have experience in Python and JavaScript. In my last project, I built a web scraper using Python libraries like BeautifulSoup and requests. I also created interactive web pages using JavaScript and React.

Your answer to Q2: When I encounter complex bugs, I first try to isolate the problem by breaking down the code and adding logging statements. For example, once I faced a memory leak in a Python app, and by using memory profiling tools, I identif